<a href="https://colab.research.google.com/github/yuanyibo666/DL_practicecode/blob/main/kaggle%E6%88%BF%E4%BB%B7%E9%A2%84%E6%B5%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **下载和缓存数据集**
+ 建立字典DATA_HUB。
+ download函数用来下载数据集， 将数据集缓存在本地目录，并且其sha-1与存储在DATA_HUB中的相匹配。
+ download_extract函数下载并解压缩一个zip或tar文件。
+ download_all函数将本书中使用的所有数据集从DATA_HUB下载到缓存目录中

In [ ]:
import hashlib
import os
import tarfile
import zipfile
import requests
import random

DATA_HUB = dict()
DATA_URL = 'http://d2l-data.s3-accelerate.amazonaws.com/'

def download(name, cache_dir=os.path.join('..', 'data')):  
    """下载一个DATA_HUB中的文件，返回本地文件名"""
    assert name in DATA_HUB, f"{name} 不存在于 {DATA_HUB}"
    url, sha1_hash = DATA_HUB[name]
    os.makedirs(cache_dir, exist_ok=True)
    fname = os.path.join(cache_dir, url.split('/')[-1])
    if os.path.exists(fname):
        sha1 = hashlib.sha1()
        with open(fname, 'rb') as f:
            while True:
                data = f.read(1048576)
                if not data:
                    break
                sha1.update(data)
        if sha1.hexdigest() == sha1_hash:
            return fname  # 命中缓存
    print(f'正在从{url}下载{fname}...')
    r = requests.get(url, stream=True, verify=True)
    with open(fname, 'wb') as f:
        f.write(r.content)
    return fname

def download_extract(name, folder=None):  
    """下载并解压zip/tar文件"""
    fname = download(name)
    base_dir = os.path.dirname(fname)
    data_dir, ext = os.path.splitext(fname)
    if ext == '.zip':
        fp = zipfile.ZipFile(fname, 'r')
    elif ext in ('.tar', '.gz'):
        fp = tarfile.open(fname, 'r')
    else:
        assert False, '只有zip/tar文件可以被解压缩'
    fp.extractall(base_dir)
    return os.path.join(base_dir, folder) if folder else data_dir

def download_all():  
    """下载DATA_HUB中的所有文件"""
    for name in DATA_HUB:
        download(name)

### **访问并读取数据集**
  使用pandas读入并处理数据，并分别加载包含训练数据和测试数据的两个CSV文件。
 
  训练数据集包括1460个样本，每个样本80个特征和1个标签， 而测试数据集包含1459个样本，每个样本80个特征。


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.optim as optim
from IPython import display
import matplotlib.pyplot as plt

In [ ]:
DATA_HUB['kaggle_house_train'] = (  
    DATA_URL + 'kaggle_house_pred_train.csv',
    '585e9cc93e70b39160e7921475f9bcd7d31219ce')

DATA_HUB['kaggle_house_test'] = (  
    DATA_URL + 'kaggle_house_pred_test.csv',
    'fa19780a7b011d9b009e8bff8e99922a8ee2eb90')
train_data = pd.read_csv(download('kaggle_house_train'))
test_data = pd.read_csv(download('kaggle_house_test'))
print(train_data.shape)
print(test_data.shape)
print(train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]])

(1460, 81)
(1459, 80)
   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000


In [ ]:
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))

### **数据预处理**
+ 将所有缺失的值替换为相应特征的平均值
+ 为了将所有特征放在一个共同的尺度上，通过将特征重新缩放到零均值和单位方差来标准化数据：
$$x\leftarrow\frac{x-\mu}{\sigma}$$
$\mu$表示均值，$\sigma$表示标准差，经标准化后，这些特征具有零均值和单位方差。
+ 用one-hot编码处理离散值。

  处理后，特征的总数量从79个增加到331个，通过value属性，从pandas格式中提取NumPy格式，并将其转换为张量表示用于训练。

In [ ]:
# 若无法获得测试数据，则可根据训练数据计算均值和标准差
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)
# “Dummy_na=True”将“na”（缺失值）视为有效的特征值，并为其创建指示符特征
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float32)
train_labels = torch.tensor(
    train_data.SalePrice.values.reshape(-1, 1), dtype=torch.float32)

In [ ]:
print(train_features)
print(train_features.shape)
print(test_features.shape)
print(train_labels.shape)

tensor([[ 0.0673, -0.1844, -0.2178,  ...,  1.0000,  0.0000,  0.0000],
        [-0.8735,  0.4581, -0.0720,  ...,  1.0000,  0.0000,  0.0000],
        [ 0.0673, -0.0559,  0.1372,  ...,  1.0000,  0.0000,  0.0000],
        ...,
        [ 0.3025, -0.1416, -0.1428,  ...,  1.0000,  0.0000,  0.0000],
        [-0.8735, -0.0559, -0.0572,  ...,  1.0000,  0.0000,  0.0000],
        [-0.8735,  0.2439, -0.0293,  ...,  1.0000,  0.0000,  0.0000]])
torch.Size([1460, 331])
torch.Size([1459, 331])
torch.Size([1460, 1])


### **生成迭代器**

  为了在训练模型时对数据集进行遍历，每次抽取一小批量样本并使用它们来更新模型。

  定义一个**data_iter**函数，该函数接受**批量大小、特征矩阵和标签向量**作为输入，每次生成大小为**batch_size**的小批量，每个小批量包含一组特征和标签。每次选取小批量时随机选取。

In [ ]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]
        # yield返回生成器，下次调用data_iter时从上次yield位置继续执行

### **定义激活函数ReLU**
ReLU函数其实是分段线性函数，把所有的负值都变为0，而正值不变，这种操作被成为单侧抑制。

ReLU函数的数学原理如下：
$$RELU(x)=\max(x,0)$$

#### 激活函数评估：
+ 优点：ReLu具有稀疏性，可以使稀疏后的模型能够更好地挖掘相关特征，拟合训练数据；在x>0区域上，不会出现梯度饱和、梯度消失的问题；计算复杂度低，不需要进行指数运算，只要一个阈值就可以得到激活值。
+ 不足：输出不是0对称；由于小于0的时候激活函数值为0，梯度为0，所以存在一部分神经元永远不会得到更新。

In [ ]:
#激活函数
def relu(X):
    a = torch.zeros_like(X)
    return torch.max(X, a)

#### **将线性网络的结果进行映射**

$$\hat{y_i}=e^{y_i}$$
+ 目的：使$\hat{y}$大于零,便于损失函数的计算。

In [ ]:
def my_exp(X):
  X_exp=torch.exp(X)
  return X_exp

### **定义MLP网络**
+ 输入层：331个节点
+ 隐藏层（1层）：256个节点
+ 输出层：1个节点

In [ ]:
#计算模型
def net(X,W1,W2,W3,b1,b2,b3):
    H1 = relu(torch.matmul(X,W1) + b1)
    H2 = relu(torch.matmul(H1,W2)+b2)
    return my_exp(torch.matmul(H2,W3) + b3)

### **定义损失函数**
房价就像股票价格一样，关心的是相对数量，而不是绝对数量。因此，**更关心相对误差$\frac{y - \hat{y}}{y}$，**
而不是绝对误差$y - \hat{y}$。

即将$\delta$ for $|\log y - \log \hat{y}| \leq \delta$
转换为$e^{-\delta} \leq \frac{\hat{y}}{y} \leq e^\delta$。
这使得预测价格的对数与真实标签价格的对数之间出现以下均方根误差：
$$\sqrt{\frac{1}{n}\sum^n_{i=1}(\log y_i-\log \hat{y}_i)^2}$$
对此，我们自定义了**CrossEntropyLoss函数**，该函数以$\hat{y}$和$y$为输入，实现以上损失计算。

In [ ]:
def CrossEntropyLoss(y_hat,y):
    X=(torch.log(y)-torch.log(y_hat))**2
    return (X.mean())**0.5

### **定义优化函数**
为了实现小批量随机梯度下降，我们定义了**sgd函数**，该函数以模型参**数集合、学习速率和批量大小**为输入。每一步更新的大小由学习速率**lr**决定，计算的损失grad是一个批量样本损失的总和，所以用批量大小batch_size来规范化损失，即**grad/batch_size**，这样损失大小就不会取决于我们对批量大小的选择。

In [ ]:
def sgd(params, lr, batch_size):
    """小批量随机梯度下降"""
    with torch.no_grad():
      for param in params:
        param.data -=lr*param.grad/batch_size
        param.grad.zero_()

### **定义超参数**

In [ ]:
num_epochs = 150
lr = 0.5
batch_size=10

### **初始化模型参数**
通过从均值为0，标准差为0.01的正态分布中采样随机数来初始化权重，并将偏置初始化为0。

其中W1、b1分别为输入层到隐藏层的权重和偏置，W2、b2分别为隐藏层到输出层的权重和偏置。

In [ ]:
#初始化模型参数,设置256个节点
W1 = torch.normal(0, 0.01, size=(331,256), requires_grad=True)
W2 = torch.normal(0, 0.01, size=(256,128),requires_grad=True)
W3 = torch.normal(0, 0.01, size=(128,1),requires_grad=True)
b1 = torch.zeros(256,requires_grad=True)
b2 = torch.zeros(128,requires_grad=True)
b3 = torch.zeros(1,requires_grad=True)
params = [W1,b1,W2,b2]
for param in params:
   param.requires_grad_(requires_grad = True)

In [ ]:
def k_fold(fold_num,train_features,train_labels):
  """
  k折交叉验证
  """
  size = len(train_features)
  lock_size = size/fold_num
  indices = list(range(size))
  for i in range(0,fold_num):
    temp_indices = torch.tensor(
            indices[i*lock_size: (i+1)*lock_size-1])
    yield train_features[temp_indices], train_labels[temp_indices]


### **模型训练**
通过训练集训练模型调整参数大小，用训练好的网络预测测试集。

执行以下的代码将生成一个名为submission.csv的文件。

In [ ]:
def train_and_pred(train_features, test_feature, train_labels, test_data,
                   num_epochs, lr,batch_size):
    # 训练
    for epoch in range(num_epochs):
      num = 0
      for x,y in data_iter(batch_size,train_features,train_labels):
        y_hat = net(x,W1,W2,W3,b1,b2,b3)
        num+=1
        l = CrossEntropyLoss(y_hat,y)
        l.sum().backward()
        sgd([W1,b1,W2,b2,W3,b3], lr, batch_size)
        if num%50 == 0:
          print('epoch %d,num %d: loss'%(epoch,num),l)
    
    # 预测    
    predss=net(test_feature,W1,W2,W3,b1,b2,b3).clone()
    preds=predss.detach().numpy()
    # 将其重新格式化以导出到Kaggle
    # preds 是你对所有训练数据的预测结果，形状应该是 (1459, 1) 或类似形状的。（训练数据个数是1459）
    test_data['SalePrice'] = pd.Series(preds.reshape(1, -1)[0])
    submission = pd.concat([test_data['Id'], test_data['SalePrice']], axis=1)
    submission.to_csv('submission3.csv', index=False)

In [ ]:

train_and_pred(train_features, test_features, train_labels, test_data,
               num_epochs, lr, batch_size)

epoch 0,num 50: loss tensor(2.5281, grad_fn=<PowBackward0>)
epoch 0,num 100: loss tensor(4.6960, grad_fn=<PowBackward0>)
epoch 1,num 50: loss tensor(3.1735, grad_fn=<PowBackward0>)
epoch 1,num 100: loss tensor(1.0850, grad_fn=<PowBackward0>)
epoch 2,num 50: loss tensor(0.3747, grad_fn=<PowBackward0>)
epoch 2,num 100: loss tensor(0.5279, grad_fn=<PowBackward0>)
epoch 3,num 50: loss tensor(0.3132, grad_fn=<PowBackward0>)
epoch 3,num 100: loss tensor(0.3984, grad_fn=<PowBackward0>)
epoch 4,num 50: loss tensor(0.4756, grad_fn=<PowBackward0>)
epoch 4,num 100: loss tensor(0.6344, grad_fn=<PowBackward0>)
epoch 5,num 50: loss tensor(0.5242, grad_fn=<PowBackward0>)
epoch 5,num 100: loss tensor(0.3957, grad_fn=<PowBackward0>)
epoch 6,num 50: loss tensor(0.3591, grad_fn=<PowBackward0>)
epoch 6,num 100: loss tensor(0.3342, grad_fn=<PowBackward0>)
epoch 7,num 50: loss tensor(0.5195, grad_fn=<PowBackward0>)
epoch 7,num 100: loss tensor(0.5526, grad_fn=<PowBackward0>)
epoch 8,num 50: loss tensor(0.54